## BIG FAT FUNCTION

In [4]:
% cd drive/My\ Drive/sber_hack

[Errno 2] No such file or directory: 'drive/My Drive/sber_hack'
/content/drive/My Drive/sber_hack


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
import tqdm

In [6]:
train_df = pd.read_csv('train.csv')
all_retailers = sorted(list(train_df['retailer'].unique()))

store_products_by_sales = pd.read_csv('store_products_by_sales.csv')
df_agg = store_products_by_sales.groupby(['retailer', 'product_id']).agg({'sales':sum})
g = df_agg['sales'].groupby(level=0, group_keys=False)
top100_retailers = g.apply(lambda x: x.sort_values(ascending=False).head(100)).reset_index().drop(['sales'], axis=1)

top100_retailers_count = g.apply(lambda x: x.sort_values(ascending=False).head(100)).reset_index()

In [ ]:
len(top100_retailers[top100_retailers.retailer == 'Home Market'])

27

In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression

In [ ]:
def build_and_train(path, retailer, top100_retailers, top100_retailers_count, mode='train', path_train=None):
    retail_data = pd.read_csv(path)

    if mode == 'test':
        train_data = pd.read_csv(path_train)

    # one hot for platform & products
    retail_data['undefined'] = np.zeros(len(retail_data))
    one_hot = pd.get_dummies(retail_data['platform'])
    retail_data[one_hot.columns] = one_hot 
    
    if mode == 'test': 
        retail_data = pd.merge(retail_data, train_data.groupby(by='user_id').order_id.max(), how='left', on='user_id')
        retail_data = pd.merge(retail_data, train_data[['order_id', 'product_id']], how='left', left_on='order_id_y', right_on='order_id')
        retail_data.product_id = retail_data.product_id.fillna('[0, 0]')
    
    top100_poducts = top100_retailers.loc[top100_retailers.retailer == retailer].product_id.unique()
    n_products = len(top100_poducts)
    retail_data.product_id = [[int(idx) for idx in x[1:-1].split(',')] for x in retail_data.product_id]

    if mode == 'train':
        ml = retail_data.join(
                              pd.DataFrame.sparse.from_spmatrix(
                              bnrs[retailer].fit_transform(retail_data.pop('product_id')),
                              index=retail_data.index,
                              columns=bnrs[retailer].classes_))
        
    elif mode == 'test':
        ml = retail_data.join(
                              pd.DataFrame.sparse.from_spmatrix(
                              bnrs[retailer].transform(retail_data.pop('product_id')),
                              index=retail_data.index,
                              columns=bnrs[retailer].classes_))
    
    
    retail_data[top100_poducts] = ml[top100_poducts].values

    if mode == 'train':
        mean_products[retailer] = retail_data[top100_poducts].values.mean(axis=0)

        retail_users_and_orders = retail_data.groupby(by='user_id').order_id.apply(list)[retail_data.groupby(by='user_id').order_id.count() > 1].reset_index()
        if len(retail_users_and_orders) < 1:
            retail_data_fake = retail_data.copy()
            retail_data_fake.order_id -= 1
            retail_data_fake[top100_poducts] = np.tile(mean_products[retailer], len(retail_data_fake)).reshape(-1, n_products)

            retail_data = retail_data.append(retail_data_fake, ignore_index=True)
            retail_users_and_orders = retail_data.groupby(by='user_id').order_id.apply(list)[retail_data.groupby(by='user_id').order_id.count() > 1].reset_index()

    # train KNN
    if mode == 'train':
        X_knn = retail_data.drop(['user_id', 'order_id', 'order_created_time', 'retailer', 'store_id', 'platform'], axis=1).values
        X_knn_scaled = sclrs_knn[retailer].fit_transform(X_knn)
        nbrs[retailer] = nbrs[retailer].fit(X_knn_scaled)
    
    elif  mode == 'test':
        retail_data.loc[retail_data.order_id.isna(), top100_poducts] = np.tile(mean_products[retailer], len(retail_data.loc[retail_data.order_id.isna()])).reshape(-1, n_products)
        X_knn = retail_data.drop(['user_id', 'order_id', 'order_id_x', 'order_id_y', 'order_created_time', 'retailer', 'store_id', 'platform'], axis=1).fillna(0).values
        X_knn_scaled = sclrs_knn[retailer].transform(X_knn)
        
    distances_retail, indices_retail = nbrs[retailer].kneighbors(X_knn_scaled)

    # create dataset for logreg
    if mode == 'train':
        logreg_data[retailer] = retail_data[top100_poducts].values
        mean_products[retailer] = logreg_data[retailer].mean(axis=0)

        train_X, train_Y = [], []
        for user, orders in tqdm.tqdm(retail_users_and_orders.values):
            train_order_x, train_order_y = np.sort(orders)[-2:]
            ind_x = retail_data.loc[(retail_data.user_id == user) & (retail_data.order_id == train_order_x)].index
            ind_y = retail_data.loc[(retail_data.user_id == user) & (retail_data.order_id == train_order_y)].index

            neighbors = indices_retail[ind_x][indices_retail[ind_x] != ind_y][:K_NEIGHBOURS[retailer]-1]
            
            train_X += [np.vstack([logreg_data[retailer][neighbors], top100_retailers_count.loc[top100_retailers_count.retailer == retailer].sales.values]).T]
            train_Y += [logreg_data[retailer][ind_y][0]]
        
        train_X, train_Y = np.vstack(train_X), np.hstack(train_Y)

    elif mode == 'test':
        train_X = []
        for inds in indices_retail:
            neighbors = inds[:K_NEIGHBOURS[retailer]-1]
            
            train_X += [np.vstack([logreg_data[retailer][neighbors], top100_retailers_count.loc[top100_retailers_count.retailer == retailer].sales.values]).T]

        train_X = np.vstack(train_X)

    # fit LogReg
    if mode == 'train':
        train_X_scaled = sclrs_lr[retailer].fit_transform(train_X)
        clfs[retailer] = clfs[retailer].fit(train_X_scaled, train_Y)

        return clfs[retailer].score(train_X_scaled, train_Y)
    
    elif mode == 'test':
        train_X_scaled = sclrs_lr[retailer].transform(train_X)
        clfs[retailer].predict_proba(train_X_scaled)

        prediction_probs = clfs[retailer].predict_proba(train_X_scaled)[:, 1].reshape(-1, n_products)
        best_args = np.argsort(-prediction_probs, axis=1)[:, :min(n_products, 50)]
        return top100_poducts[best_args]

In [ ]:
path = '/content/drive/My Drive/sber_hack/by_retailers/'

bnrs, sclrs_knn, sclrs_lr, nbrs, clfs  = {}, {}, {}, {}, {}
mean_products, logreg_data, n_samples, K_NEIGHBOURS = {}, {}, {}, {}

for retailer in all_retailers:
    retail_data = pd.read_csv(path + retailer + '/train_' + retailer + '.csv')
    n_samples[retailer] = len(retail_data)
    
    bnrs[retailer] = MultiLabelBinarizer(sparse_output=True)
    sclrs_knn[retailer] = StandardScaler()
    sclrs_lr[retailer] = StandardScaler()

    K_NEIGHBOURS[retailer] = 100
    if n_samples[retailer] < 100:
        K_NEIGHBOURS[retailer] = n_samples[retailer]

    nbrs[retailer] = NearestNeighbors(n_neighbors=K_NEIGHBOURS[retailer], algorithm='ball_tree')
    clfs[retailer] = LogisticRegression(random_state=0)

    mean_products[retailer] = 0
    logreg_data[retailer] = []

In [ ]:
path = '/content/drive/My Drive/sber_hack/by_retailers/'

for retailer in all_retailers:
    accuracy = build_and_train(path + retailer + '/train_' + retailer + '.csv', retailer, top100_retailers, top100_retailers_count, mode='train')
    print('Predictions on train for retailer:', retailer, 'gained accuracy =', accuracy)

100%|██████████| 4/4 [00:00<00:00, 196.88it/s]

Predictions on train for retailer: BILLA gained accuracy = 0.9535779816513762
Predictions on train for retailer: Home Market gained accuracy = 0.7314814814814815


In [ ]:
for retailer in tqdm.tqdm(all_retailers):
    retail_data = pd.read_csv(path + retailer + '/train_' + retailer + '.csv')
    print(retailer, len(retail_data))

  0%|          | 0/45 [00:00<?, ?it/s]

BILLA 1908
Home Market 4


  7%|▋         | 3/45 [00:10<02:30,  3.58s/it]

METRO 1222655


  9%|▉         | 4/45 [00:11<01:57,  2.86s/it]

...

Хороший 321


 98%|█████████▊| 44/45 [00:54<00:00,  1.25it/s]

Шан 10


100%|██████████| 45/45 [00:54<00:00,  1.22s/it]

ЮгОпт 12


In [ ]:
prediction = build_and_train('/content/drive/My Drive/sber_hack/by_retailers/BILLA/test_BILLA.csv', 'BILLA', 
                              top100_retailers, top100_retailers_count, mode='test',
                              path_train='/content/drive/My Drive/sber_hack/by_retailers/BILLA/train_BILLA.csv')

prediction.shape

(137, 50)